In [1]:
# (código = 8) Dispensa de Licitação
# (código = 6) Pregão - Eletrônico

In [2]:

#Imports e configurações iniciais

import requests
import sqlite3
import json
import time
from datetime import datetime

# --- Configurações Base ---
BASE_URL = "https://pncp.gov.br/api/consulta" #URL BASE
ENDPOINT = "/v1/contratacoes/proposta" #ENDPOINT p/ consultar contratações com período de recebimento em aberto

URL_FINAL = BASE_URL + ENDPOINT

# Parâmetros da requisição
CODIGO_MODALIDADE = 6 # 6 = Dispensa de Licitação, conforme sua solicitação
DATA_FINAL = datetime.now().strftime('%Y%m%d') # Data de hoje no formato AAAAMMDD

print(DATA_FINAL)

# Configurações do Banco de dados
NOME_DB = "pncp_data.db"

20251029


In [3]:

# Parâmetros obrigatórios e recomendados para a consulta
import datetime # Certifique-se que esta linha está no topo do seu script!

# --- Parâmetros da Requisição ---

# Parâmetros obrigatórios e recomendados para a consulta
params = {
    #'dataFinal': datetime.date.today().strftime('%Y%m%d'), # Data final (hoje) no formato AAAAMMDD

    'dataFinal': 20251130,
    'codigoModalidadeContratacao': 6, # 6 = Pregão
    'pagina': 1, # Número da página que se deseja obter
    'tamanhoPagina': 50 # Na documentação o limite é 500, porém só funciona 50
}

# No loop, a atualização da página é feita da seguinte maneira:
# params['pagina'] = pagina_atual

In [4]:
def configurar_db():
    """
    Cria o banco de dados SQLite e a tabela com Colunas Geradas para Indexação JSON1.
    
    ATENÇÃO: Para aplicar esta nova coluna, você DEVE excluir o arquivo
    pncp_data.db existente antes de rodar esta função.
    """
    try:
        conn = sqlite3.connect(NOME_DB)
        cursor = conn.cursor()
        
        print(f"Configurando banco de dados: {NOME_DB}")
        
        # 1. Criar a tabela com Colunas Geradas (JSON1)
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS contratacoes (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                
                -- Colunas de Controle
                pagina_coleta INTEGER,
                timestamp_coleta TEXT,
                
                -- Coluna Principal para o JSON Bruto
                dados_json TEXT, 
                
                -- Colunas Geradas (Indexáveis)
                pncp_id TEXT AS (json_extract(dados_json, '$.numeroControlePNCP')) STORED,
                orgao_nome TEXT AS (json_extract(dados_json, '$.orgaoEntidade.razaosocial')) STORED,
                uf_sigla TEXT AS (json_extract(dados_json, '$.unidadeOrgao.ufSigla')) STORED,
                
                -- *** NOVA COLUNA GERADA PARA ML ***
                objeto_compra TEXT AS (json_extract(dados_json, '$.objetoCompra')) STORED,
                
                -- Garante que não haja duplicatas
                UNIQUE(pncp_id) 
            );
        """)

        # 2. Criar Índices nas colunas geradas para performance de busca
        cursor.execute("CREATE INDEX IF NOT EXISTS idx_pncp_id ON contratacoes (pncp_id);")
        cursor.execute("CREATE INDEX IF NOT EXISTS idx_orgao_nome ON contratacoes (orgao_nome);")
        cursor.execute("CREATE INDEX IF NOT EXISTS idx_uf_sigla ON contratacoes (uf_sigla);")
        
        # *** NOVO ÍNDICE PARA O CAMPO DE ML ***
        cursor.execute("CREATE INDEX IF NOT EXISTS idx_objeto_compra ON contratacoes (objeto_compra);")
        
        conn.commit()
        conn.close()
        print("Banco de dados, tabela e índices configurados com sucesso.")
    except sqlite3.Error as e:
        print(f"ERRO ao configurar o DB: {e}")

# Executar a função de setup (garantir que ela seja executada antes do loop)
configurar_db()

Configurando banco de dados: pncp_data.db
Banco de dados, tabela e índices configurados com sucesso.


In [5]:
def inserir_dados(nome_db, dados_pagina, pagina_coletada):
    """Insere uma lista de registros (uma página) no SQLite."""
    try:
        conn = sqlite3.connect(nome_db)
        cursor = conn.cursor()
        
        # Correção do Bug de Import: Usa datetime.datetime.now() ou ajuste o import
        # (Assumindo que você ajustou o import para apenas 'import datetime')
        timestamp_atual = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S') 
        
        registros_inseridos = 0
        
        for registro in dados_pagina:
            registro_json_str = json.dumps(registro)
            
            # O INSERT utiliza as novas colunas de controle (pagina_coleta, timestamp_coleta)
            # e a coluna principal (dados_json). As colunas geradas são populadas 
            # automaticamente pelo SQLite.
            cursor.execute("""
                INSERT OR IGNORE INTO contratacoes (pagina_coleta, timestamp_coleta, dados_json)
                VALUES (?, ?, ?);
            """, (pagina_coletada, timestamp_atual, registro_json_str))
            
            # Contamos apenas os registros que foram inseridos (o IGNORE pula duplicatas)
            if cursor.rowcount > 0:
                registros_inseridos += 1
            
        conn.commit()
        print(f"   -> [DB] {registros_inseridos}/{len(dados_pagina)} registros salvos/atualizados na página {pagina_coletada}.")
        
    except sqlite3.Error as e:
        print(f"   -> [ERRO DB] Falha ao inserir dados: {e}. Verifique se o arquivo DB tem a estrutura correta.")
    finally:
        if conn:
            conn.close()

In [6]:
def fetch_com_retry(url, params, max_tentativas=5, backoff_inicial=1):
    """Busca dados da API com retry e backoff exponencial."""
    
    headers = {'accept': '*/*'}
    tentativas = 0
    backoff = backoff_inicial
    
    while tentativas < max_tentativas:
        try:
            resposta = requests.get(url, params=params, headers=headers, timeout=30)
            
            # 200 OK: Sucesso
            if resposta.status_code == 200:
                print("Código de resposta 200 (OK)")
                dados_resposta = resposta.json()
                
                # *** ATUALIZAÇÃO ESSENCIAL AQUI ***
                if isinstance(dados_resposta, dict) and 'data' in dados_resposta:
                    
                    # Extraímos a lista de dados E o total de páginas da resposta
                    dados_lista = dados_resposta.get('data', [])
                    
                    # Assumindo que a API retorna 'totalPaginas' no mesmo nível do 'data'
                    # Se o nome da chave for outro (ex: 'total_paginas'), ajuste aqui.
                    total_paginas = dados_resposta.get('totalPaginas') 
                    
                    # Retornamos uma TUPLA (dados, total_paginas)
                    return (dados_lista, total_paginas)
                # --- FIM DA ATUALIZAÇÃO ---
                
                else:
                    # 200 OK com formato inesperado
                    print("Resposta 200 OK, mas JSON em formato inesperado ou lista vazia.")
                    return ([], None) # Retorna tupla (lista vazia, sem total)
            
            # 204 No Content: Fim dos dados/paginação 
            if resposta.status_code == 204:
                print("Status 204 (No Content). Fim dos dados.")
                return ([], None) # Retorna tupla (lista vazia, sem total)
            
            # 400/422: Erro nos parâmetros, não adianta tentar de novo 
            if resposta.status_code in [400, 422]:
                print(f"Erro de cliente {resposta.status_code}: {resposta.text}")
                return None # Abortar (erro)
            
            # 500: Erro de servidor, vamos tentar de novo 
            print(f"Erro {resposta.status_code}. Tentando novamente em {backoff}s...")
        
        except requests.exceptions.RequestException as e:
            print(f"Erro de conexão: {e}. Tentando novamente em {backoff}s...")
        except json.JSONDecodeError:
            print("Erro ao decodificar JSON. Retornando None.")
            return None
        
        time.sleep(backoff)
        tentativas += 1
        backoff *= 2 # Backoff exponencial
        
    print("Número máximo de tentativas atingido. Falha ao buscar dados.")
    return None

In [7]:
teste = fetch_com_retry(URL_FINAL, params)

print(teste)

json_formatado = json.dumps(teste, indent=4, ensure_ascii=False)

print(json_formatado)


Código de resposta 200 (OK)
([{'srp': False, 'orgaoEntidade': {'cnpj': '04892707000100', 'razaoSocial': 'DNIT-DEPARTAMENTO NACIONAL DE INFRAEST DE TRANSPORTES', 'poderId': 'E', 'esferaId': 'F'}, 'anoCompra': 2024, 'sequencialCompra': 687, 'dataInclusao': '2024-08-12T07:02:46', 'dataPublicacaoPncp': '2024-08-12T07:02:46', 'dataAtualizacao': '2025-10-17T07:39:59', 'numeroCompra': '90266', 'unidadeOrgao': {'ufNome': 'Pará', 'codigoUnidade': '393016', 'nomeUnidade': 'SUPERINTENDêNCIA REGIONAL NO ESTADO DO PARA', 'ufSigla': 'PA', 'municipioNome': 'Belém', 'codigoIbge': '1501402'}, 'amparoLegal': {'descricao': 'pregão: modalidade de licitação obrigatória para aquisição de bens e serviços comuns', 'nome': 'Lei 14.133/2021, Art. 28, I', 'codigo': 1}, 'dataAberturaProposta': '2025-10-17T08:00:00', 'dataEncerramentoProposta': '2025-11-06T10:00:00', 'informacaoComplementar': 'Edital disponível no Portal do DNIT e da Divulgação de Compras Para as respostas de esclarecimentos e impugnações deste ed

In [8]:
def buscar_dados_paginados(url_completa, parametros):
    """
    Busca todos os dados da API PNCP usando paginação e salva no DB a cada iteração.
    (Versão atualizada para mostrar o progresso X de Y)
    """
    
    pagina_atual = 1
    total_registros_processados = 0
    
    # --- NOVO: Variável para armazenar o total de páginas ---
    total_paginas_api = '???' # Começa como desconhecido
    
    print("--- INICIANDO BUSCA PAGINADA ---")
    
    while True:
        # 1. Preparar parâmetros (Sem mudanças)
        params_paginados = parametros.copy()
        params_paginados['pagina'] = pagina_atual 
        
        # --- ATUALIZAÇÃO: Mostra o progresso (X de Y) ---
        print(f"\n>>>> BUSCANDO PÁGINA: {pagina_atual} de {total_paginas_api}")
        
        # 2. Chamar a função de busca 
        # --- ATUALIZAÇÃO: Espera uma tupla (dados, total_paginas) ou None ---
        retorno_fetch = fetch_com_retry(url_completa, params_paginados)
        
        # 3. Verificar condição de parada: None (erro)
        if retorno_fetch is None:
            print(f"Fim dos dados ou erro (fetch_com_retry retornou None) na página {pagina_atual}. FIM DA BUSCA.")
            break 
            
        # --- ATUALIZAÇÃO: Desempacotar a tupla ---
        dados_pagina, total_paginas_resposta = retorno_fetch
        
        # --- ATUALIZAÇÃO: Atualizar o total de páginas se ele for informado ---
        # (Só atualiza na primeira vez ou se mudar)
        if total_paginas_resposta is not None:
            total_paginas_api = total_paginas_resposta
        # --- FIM DAS ATUALIZAÇÕES ---

        # 3. (Continuação) Verificar condição de parada: lista vazia ([])
        # (Agora checamos 'dados_pagina' após desempacotar)
        if not dados_pagina: # Checa se a lista de dados está vazia
            print(f"Fim dos dados (lista vazia recebida) na página {pagina_atual}. FIM DA BUSCA.")
            break 
            
        # 4. PERSISTÊNCIA: Salvar no Banco de Dados (Sem mudanças)
        inserir_dados(NOME_DB, dados_pagina, pagina_atual)
        total_registros_processados += len(dados_pagina)

        # 5. Visualizar a página (em tempo real)
        if dados_pagina: 
            # --- ATUALIZAÇÃO: Inclui o total de páginas no log ---
            print(f"Dados recebidos (Página {pagina_atual}/{total_paginas_api}, {len(dados_pagina)} registros):")
            
            # (Lógica original mantida)
            json_formatado = json.dumps(dados_pagina[:3], indent=4, ensure_ascii=False)
            print(json_formatado) 
            print("   -> (JSON truncado, salvamento completo e indexado no DB)")

        # 6. Incrementar e prosseguir (Sem mudanças)
        pagina_atual += 1
        time.sleep(0.5) # Pausa recomendada
        
    print("\n--- BUSCA CONCLUÍDA ---")
    print(f"Total de registros processados: {total_registros_processados}")
    return True

In [9]:
buscar_dados_paginados(URL_FINAL, params)

--- INICIANDO BUSCA PAGINADA ---

>>>> BUSCANDO PÁGINA: 1 de ???
Código de resposta 200 (OK)
   -> [DB] 50/50 registros salvos/atualizados na página 1.
Dados recebidos (Página 1/373, 50 registros):
[
    {
        "srp": false,
        "orgaoEntidade": {
            "cnpj": "04892707000100",
            "poderId": "E",
            "esferaId": "F",
            "razaoSocial": "DNIT-DEPARTAMENTO NACIONAL DE INFRAEST DE TRANSPORTES"
        },
        "anoCompra": 2024,
        "sequencialCompra": 687,
        "dataInclusao": "2024-08-12T07:02:46",
        "dataPublicacaoPncp": "2024-08-12T07:02:46",
        "dataAtualizacao": "2025-10-17T07:39:59",
        "numeroCompra": "90266",
        "unidadeOrgao": {
            "ufNome": "Pará",
            "codigoUnidade": "393016",
            "nomeUnidade": "SUPERINTENDêNCIA REGIONAL NO ESTADO DO PARA",
            "ufSigla": "PA",
            "municipioNome": "Belém",
            "codigoIbge": "1501402"
        },
        "amparoLegal": {
     

True

In [10]:
# --- CÉLULA DE ML 1: IMPORTS (VERSÃO DEFINITIVA SIMPLIFICADA) ---
# Focamos em bibliotecas que não dependem de downloads externos que falham.

print("Carregando bibliotecas de ML (Versão Simplificada)...")

# --- Imports Essenciais ---
import pandas as pd
import numpy as np
import re
import joblib 

# --- Imports de NLTK: Usaremos  as stopwords ---
import nltk
from nltk.corpus import stopwords
# Removemos a importação de word_tokenize e PunktLanguageVars/PunktSentenceTokenizer

# --- Imports do Scikit-learn (sklearn) para o Pipeline de ML ---
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

print("Bibliotecas de ML importadas com sucesso.")

# --- Configuração de Stopwords ---
print(f"\n--- Preparando Stopwords em Português---")

try:
    # Carrega a lista de stopwords específicas do idioma 'portuguese'.
    lista_stopwords = stopwords.words('portuguese')
except LookupError:
     # Fallback seguro para o caso de falha TOTAL do NLTK
     print("ERRO: Stopwords não carregadas. Usando lista mínima de fallback.")
     lista_stopwords = ['a', 'o', 'e', 'de', 'para', 'com', 'um', 'uma']

# Converte a LISTA em um SET (conjunto) para busca rápida.
stop_words_pt = set(lista_stopwords)

print(f"Total de stopwords (SET) carregadas: {len(stop_words_pt)}")
print("--- Célula 10 concluída: Ferramentas prontas. Otimização final será feita na Célula 11. ---")

Carregando bibliotecas de ML (Versão Simplificada)...
Bibliotecas de ML importadas com sucesso.

--- Preparando Stopwords em Português---
ERRO: Stopwords não carregadas. Usando lista mínima de fallback.
Total de stopwords (SET) carregadas: 8
--- Célula 10 concluída: Ferramentas prontas. Otimização final será feita na Célula 11. ---


In [11]:
# --- CÉLULA DE ML 2: FUNÇÃO DE LIMPEZA DE TEXTO (VERSÃO DEFINITIVA SIMPLIFICADA) ---
# Elimina a dependência de NLTK word_tokenize (punkt/punkt_tab) usando Regex para tokenizar.

# Definição da função 'preprocessar_texto'. Ela recebe uma string 'texto'.
def preprocessar_texto(texto):

    # 'verbose' controla se a função imprime seu passo-a-passo.
    verbose = True if 'exemplo_teste' in globals() else False 

    if verbose: print(f"\n--- INICIANDO 'preprocessar_texto' ---")
    if verbose: print(f"0. DADO ORIGINAL: '{texto}'")

    # ETAPA 1: Verificação de Segurança (Tipo)
    if not isinstance(texto, str):
        return "" 

    # ETAPA 2: Normalização (Minúsculas)
    texto_etapa2 = texto.lower()
    if verbose: print(f"\n1. APÓS .lower(): '{texto_etapa2}'")

    # ETAPA 3: Limpeza de Ruído (Regex)
    # Remove tudo que não for letra (a-z) ou acentuação (À-ú)
    texto_etapa3 = re.sub(r'[^a-zÀ-ú\s]', ' ', texto_etapa2, flags=re.IGNORECASE)
    # Remove múltiplos espaços
    texto_etapa3 = re.sub(r'\s+', ' ', texto_etapa3).strip()
    if verbose: print(f"\n2. APÓS REGEX (remover números/pontuação/multi-espaços): '{texto_etapa3}'")

    # ETAPA 4: Tokenização (QUEBRAR em palavras usando Regex, SEM NLTK PUNKT)
    # Encontra todas as sequências de letras.
    # ISSO SUBSTITUI O word_tokenize PROBLEMÁTICO
    tokens_etapa4 = re.findall(r'\b[a-zÀ-ú]+\b', texto_etapa3)
    if verbose: print(f"\n3. APÓS TOKENIZAÇÃO (Regex): {tokens_etapa4}")

    # ETAPA 5: Remoção de Stopwords e Palavras Curtas
    # 'stop_words_pt' deve estar carregada da Célula 10
    tokens_etapa5 = [palavra for palavra in tokens_etapa4 if palavra not in stop_words_pt and len(palavra) > 2]
    if verbose: print(f"\n4. APÓS FILTRAR stopwords e palavras curtas: {tokens_etapa5}")

    # ETAPA 6: Remontagem da String
    texto_final_etapa6 = " ".join(tokens_etapa5)
    if verbose: print(f"\n5. APÓS .join() (virou STRING de novo): '{texto_final_etapa6}'")
    if verbose: print(f"--- FIM 'preprocessar_texto' (Resultado Final: {texto_final_etapa6}) ---")

    return texto_final_etapa6

# --- Teste Rápido ---
print("\n--- TESTANDO A FUNÇÃO 'preprocessar_texto' ---")
exemplo_teste = "Pregão Eletrônico para Aquisição de 10 (dez) notebooks i7."

texto_processado = preprocessar_texto(exemplo_teste)

del exemplo_teste

print(f"\n===================================================\nRESULTADO FINAL DO TESTE:")
print(f"  ORIGINAL: 'Pregão Eletrônico para Aquisição de 10 (dez) notebooks i7.'")
print(f"  PROCESSADO: '{texto_processado}'")
print(f"===================================================\n")


# --- Redefinição da função para modo "silencioso" (PRODUÇÃO) ---
# Esta é a versão que será usada nas Células ML 3 e 4
def preprocessar_texto(texto):

    # Modo silencioso
    if not isinstance(texto, str):
        return ""

    texto_etapa2 = texto.lower()
    texto_etapa3 = re.sub(r'[^a-zÀ-ú\s]', ' ', texto_etapa2, flags=re.IGNORECASE)
    texto_etapa3 = re.sub(r'\s+', ' ', texto_etapa3).strip()

    # Tokenização baseada em Regex (solução definitiva)
    tokens_etapa4 = re.findall(r'\b[a-zÀ-ú]+\b', texto_etapa3)

    # Remoção de Stopwords e Palavras Curtas
    tokens_etapa5 = [palavra for palavra in tokens_etapa4 if palavra not in stop_words_pt and len(palavra) > 2]
    
    return " ".join(tokens_etapa5)

print("Função 'preprocessar_texto' redefinida usando Regex para tokenização (Solução Definitiva).")
print("--- Célula 11 corrigida e concluída. Prossiga para a Célula 3 (Treinamento) ---")


--- TESTANDO A FUNÇÃO 'preprocessar_texto' ---

--- INICIANDO 'preprocessar_texto' ---
0. DADO ORIGINAL: 'Pregão Eletrônico para Aquisição de 10 (dez) notebooks i7.'

1. APÓS .lower(): 'pregão eletrônico para aquisição de 10 (dez) notebooks i7.'

2. APÓS REGEX (remover números/pontuação/multi-espaços): 'pregão eletrônico para aquisição de dez notebooks i'

3. APÓS TOKENIZAÇÃO (Regex): ['pregão', 'eletrônico', 'para', 'aquisição', 'de', 'dez', 'notebooks', 'i']

4. APÓS FILTRAR stopwords e palavras curtas: ['pregão', 'eletrônico', 'aquisição', 'dez', 'notebooks']

5. APÓS .join() (virou STRING de novo): 'pregão eletrônico aquisição dez notebooks'
--- FIM 'preprocessar_texto' (Resultado Final: pregão eletrônico aquisição dez notebooks) ---

RESULTADO FINAL DO TESTE:
  ORIGINAL: 'Pregão Eletrônico para Aquisição de 10 (dez) notebooks i7.'
  PROCESSADO: 'pregão eletrônico aquisição dez notebooks'

Função 'preprocessar_texto' redefinida usando Regex para tokenização (Solução Definitiva).
-

In [12]:
# --- CÉLULA DE ML 3: TREINAMENTO DO MODELO (VERSÃO CORRIGIDA) ---
# O nome das colunas foi ajustado para corresponder ao que está no dataset.csv (Objeto e Relevante).

# --- ETAPA 1: DEFINIR NOMES DE ARQUIVOS ---

# Define o nome do arquivo CSV que você criou (o "gabarito").
NOME_ARQUIVO_DATASET = "dataset.csv"
# Define o nome do arquivo onde o modelo treinado será salvo.
NOME_ARQUIVO_MODELO = "modelo_relevancia.joblib"

print(f"--- INICIANDO TREINAMENTO ---")
print(f"Arquivo de Dataset (Gabarito): '{NOME_ARQUIVO_DATASET}'")
print(f"Arquivo de Saída (Modelo): '{NOME_ARQUIVO_MODELO}'")

# 'try...except' é um bloco de segurança. Ele tenta rodar o código.
# Se o 'dataset.csv' não for encontrado, ele pula para o bloco 'except' no final.
try:
    # --- ETAPA 2: CARREGAR O DATASET ---
    print(f"\n--- ETAPA 2: Carregando '{NOME_ARQUIVO_DATASET}' para um DataFrame pandas ---")
    
    # pd.read_csv lê o arquivo e o carrega na memória como um DataFrame (tabela).
    df_treino = pd.read_csv(NOME_ARQUIVO_DATASET)
    
    # Vamos inspecionar o DataFrame que acabamos de carregar:
    print(f"Tipo da variável 'df_treino': {type(df_treino)}")
    print(f"Formato (shape) do DataFrame (linhas, colunas): {df_treino.shape}")
    print(f"Total de {len(df_treino)} exemplos (licitações) carregados.")
    print("Amostra dos dados carregados (primeiras 5 linhas):")
    # .head() mostra as 5 primeiras linhas do DataFrame.
    print(df_treino.head())

    # --- CORREÇÃO APLICADA AQUI ---
    # Usando os nomes das colunas com a primeira letra MAIÚSCULA.
    print("\nContagem de valores na coluna 'Relevante' (para ver o balanceamento):")
    # .value_counts() mostra quantos '0's e '1's nós temos.
    print(df_treino['Relevante'].value_counts())
    
    # --- ETAPA 3: PRÉ-PROCESSAMENTO (LIMPEZA DE TEXTO) ---
    print(f"\n--- ETAPA 3: Aplicando pré-processamento (função 'preprocessar_texto') ---")
    
    # Aplicando em 'Objeto' (corrigido)
    df_treino['objeto_limpo'] = df_treino['Objeto'].apply(preprocessar_texto)
    
    # Vamos inspecionar o DataFrame DEPOIS da transformação:
    print("DataFrame atualizado com a coluna 'objeto_limpo'. Amostra:")
    # Acessa a coluna original 'Objeto' (corrigido)
    print(df_treino[['Objeto', 'objeto_limpo']].head())

    # --- ETAPA 4: SEPARAÇÃO DE FEATURES (X) E LABELS (y) ---
    print(f"\n--- ETAPA 4: Separando Features (X) e Labels (y) ---")
    
    # 'X' (Features): coluna 'objeto_limpo'.
    X = df_treino['objeto_limpo']
    
    # 'y' (Label/Target): coluna 'Relevante' (corrigido).
    y = df_treino['Relevante']
    
    # Inspecionando X e y:
    print(f"Tipo da variável 'X' (Features): {type(X)}")
    print(f"Tipo da variável 'y' (Labels): {type(y)}")
    print("\nAmostra de 'X' (o que o modelo vai ler):")
    print(X.head())
    print("\nAmostra de 'y' (o que o modelo vai tentar prever):")
    print(y.head())

    # --- ETAPA 5: DIVISÃO EM TREINO E TESTE ---
    print(f"\n--- ETAPA 5: Dividindo dados em Treino e Teste (80% / 20%) ---")
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    # Inspecionando os 4 conjuntos de dados:
    print(f"Formato de 'X_train' (dados de treino): {X_train.shape}")
    print(f"Formato de 'y_train' (respostas de treino): {y_train.shape}")
    print(f"Formato de 'X_test' (dados de teste): {X_test.shape}")
    print(f"Formato de 'y_test' (respostas de teste): {y_test.shape}")

    # --- ETAPA 6: DEFINIÇÃO DO PIPELINE DE ML ---
    print(f"\n--- ETAPA 6: Definindo o Pipeline do Modelo ---")
    
    modelo_pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=1000, ngram_range=(1, 2))),
        ('clf', SGDClassifier(loss='hinge', random_state=42, max_iter=100))
    ])
    
    print("Objeto Pipeline criado com sucesso:")
    print(modelo_pipeline)

    # --- ETAPA 7: TREINAMENTO DO MODELO ---
    print(f"\n--- ETAPA 7: Treinando o modelo (método .fit()) ---")
    
    modelo_pipeline.fit(X_train, y_train)
    
    print("TREINAMENTO CONCLUÍDO.")

    # --- ETAPA 8: AVALIAÇÃO DO MODELO ---
    print(f"\n--- ETAPA 8: Avaliando o modelo nos dados de TESTE ---")
    
    y_pred = modelo_pipeline.predict(X_test)
    
    print(f"Tipo da variável 'y_pred' (Predições): {type(y_pred)}")
    print(f"Formato (shape) de 'y_pred': {y_pred.shape}")
    print("\nAmostra das predições vs. dados reais (primeiras 15):")
    print(f"  Predições (y_pred): {y_pred[:15]}")
    print(f"  Reais (y_test):     {y_test.values[:15]}")

    acuracia = accuracy_score(y_test, y_pred)
    print(f"\n==> Acurácia: {acuracia * 100:.2f}% <==")
    print("\nRelatório de Classificação Detalhado:")
    
    print(classification_report(y_test, y_pred, target_names=['Irrelevante (0)', 'Relevante (1)']))

    # --- ETAPA 9: SALVANDO O MODELO ---
    print(f"\n--- ETAPA 9: Salvando o modelo treinado em disco ---")
    
    joblib.dump(modelo_pipeline, NOME_ARQUIVO_MODELO)
    
    print(f"\nModelo salvo com sucesso como '{NOME_ARQUIVO_MODELO}'")
    print("--- Célula 12 (Treinamento) corrigida e concluída ---")

except FileNotFoundError:
    print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    print(f"ERRO CRÍTICO: Arquivo '{NOME_ARQUIVO_DATASET}' não encontrado.")
    print(f"Por favor, crie este arquivo (PASSO 1) e rode esta célula novamente.")
    print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
except Exception as e:
    # Captura qualquer outro erro inesperado
    print(f"Ocorreu um erro inesperado: {e}")

--- INICIANDO TREINAMENTO ---
Arquivo de Dataset (Gabarito): 'dataset.csv'
Arquivo de Saída (Modelo): 'modelo_relevancia.joblib'

--- ETAPA 2: Carregando 'dataset.csv' para um DataFrame pandas ---
Tipo da variável 'df_treino': <class 'pandas.core.frame.DataFrame'>
Formato (shape) do DataFrame (linhas, colunas): (74, 2)
Total de 74 exemplos (licitações) carregados.
Amostra dos dados carregados (primeiras 5 linhas):
                                              Objeto  Relevante
0  Contratação de serviços de calibração certific...          1
1  Registro de Preço para aquisição de hidrômetro...          1
2  Registro de Preço para fornecimento futuro e e...          1
3  Seleção Pública de Fornecedores para Aquisição...          1
4  Contratação de empresa especializada para aqui...          1

Contagem de valores na coluna 'Relevante' (para ver o balanceamento):
Relevante
1    74
Name: count, dtype: int64

--- ETAPA 3: Aplicando pré-processamento (função 'preprocessar_texto') ---
DataFra

In [13]:
# --- CÉLULA DE ML 4: APLICANDO O MODELO NO BANCO DE DADOS ---
# Esta célula é a "fase de produção"
# 1. Carregar o modelo treinado (o .joblib) da Célula 3.
# 2. Conectar ao seu banco de dados SQLite (o .db) da coleta original.
# 3. Carregar TODOS os dados do banco para um DataFrame.
# 4. Aplicar a MESMA função de limpeza da Célula 2 nos dados do banco.
# 5. Usar o modelo carregado para PREVER quais são relevantes.
# 6. Salvar um novo CSV SÓ com as licitações relevantes.

# --- ETAPA 1: DEFINIR NOMES DE ARQUIVOS ---

# Nome do modelo que foi SALVO na Célula 3.
NOME_ARQUIVO_MODELO = "modelo_relevancia.joblib"
# Nome do banco de dados que seu script de coleta criou (PASSO 2).
NOME_DB = "pncp_data.db"
# Nome do arquivo CSV final que conterá APENAS os itens relevantes.
ARQUIVO_SAIDA_FILTRADO = "licitacoes_filtradas.csv"

print(f"--- INICIANDO APLICAÇÃO DO MODELO ---")
print(f"Modelo a ser carregado: '{NOME_ARQUIVO_MODELO}'")
print(f"Banco de dados de origem: '{NOME_DB}'")
print(f"Arquivo de saída: '{ARQUIVO_SAIDA_FILTRADO}'")

# 'try...except' para garantir que o arquivo de modelo exista.
try:
    # --- ETAPA 2: CARREGAR O MODELO TREINADO ---
    print(f"\n--- ETAPA 2: Carregando modelo '{NOME_ARQUIVO_MODELO}' do disco ---")
    
    # 'joblib.load' "descongela" o arquivo .joblib de volta para uma variável.
    # 'modelo_carregado' agora é o nosso Pipeline (Tfidf + Classificador)
    # exatamente como ele estava ao final do treino.
    modelo_carregado = joblib.load(NOME_ARQUIVO_MODELO)
    
    # Inspecionando o modelo que carregamos:
    print("Modelo carregado com sucesso.")
    print(f"Tipo da variável 'modelo_carregado': {type(modelo_carregado)}")
    print("Conteúdo do modelo (o Pipeline):")
    print(modelo_carregado)

    # --- ETAPA 3: CARREGAR DADOS DO BANCO SQLITE ---
    print(f"\n--- ETAPA 3: Conectando e lendo dados do SQLite '{NOME_DB}' ---")
    
    # Cria uma conexão com o arquivo de banco de dados SQLite.
    conn = sqlite3.connect(NOME_DB)
    
    # Define a consulta SQL que queremos executar.
    # Vamos pegar SÓ o 'pncp_id' (para identificar) e o 'objeto_compra' (para classificar).
    # 'objeto_compra' é a coluna que seu script original criou usando JSON1.
    query_sql = "SELECT pncp_id, objeto_compra FROM contratacoes"
    print(f"Executando consulta SQL: \"{query_sql}\"")
    
    # 'pd.read_sql_query' executa a consulta no banco (via 'conn')
    # e já carrega os resultados DIRETAMENTE para um DataFrame pandas.
    df_db = pd.read_sql_query(query_sql, conn)
    
    # Fecha a conexão com o banco de dados.
    conn.close()
    print("Conexão com o banco fechada.")
    
    # Inspecionando o DataFrame carregado do banco:
    print(f"\nDados carregados do banco com sucesso.")
    print(f"Tipo da variável 'df_db': {type(df_db)}")
    print(f"Formato (shape) do DataFrame (linhas, colunas): {df_db.shape}")
    print(f"Total de {len(df_db)} licitações carregadas do banco.")
    print("Amostra dos dados do banco (primeiras 5 linhas):")
    print(df_db.head())

    # --- ETAPA 4: APLICAÇÃO DO MODELO (SE HOUVER DADOS) ---
    
    # Verifica se o DataFrame não está vazio.
    if len(df_db) > 0:
        # --- 4a. PRÉ-PROCESSAMENTO (LIMPEZA) ---
        # Temos que aplicar EXATAMENTE a mesma limpeza que fizemos nos dados de treino.
        print(f"\n--- ETAPA 4a: Limpando os {len(df_db)} textos do banco de dados... (Isso pode demorar um pouco) ---")
        
        # Cria a nova coluna 'objeto_limpo' aplicando a função da Célula 2.
        # (A função 'preprocessar_texto' deve estar definida e ter sido rodada)
        df_db['objeto_limpo'] = df_db['objeto_compra'].apply(preprocessar_texto)
        
        print("Limpeza concluída. Amostra do DataFrame com a nova coluna 'objeto_limpo':")
        print(df_db[['objeto_compra', 'objeto_limpo']].head())

        # --- 4b. PREDIÇÃO (CLASSIFICAÇÃO) ---
        print(f"\n--- ETAPA 4b: Aplicando o modelo (.predict()) para classificar todos os {len(df_db)} itens... ---")
        
        # Agora a mágica: usamos o .predict() do modelo carregado.
        # O pipeline vai AUTOMATICAMENTE:
        # 1. Pegar a coluna 'objeto_limpo' (texto limpo).
        # 2. Aplicar o Tfidf (transformar em números) que ele aprendeu no treino.
        # 3. Aplicar o Classificador (prever 0 ou 1) que ele aprendeu no treino.
        # O resultado é uma lista de 0s e 1s.
        predicoes = modelo_carregado.predict(df_db['objeto_limpo'])
        
        # Inspecionando as predições
        print("Predição concluída.")
        print(f"Tipo da variável 'predicoes': {type(predicoes)}")
        print(f"Formato (shape) das predições: {predicoes.shape}")
        print(f"Amostra das primeiras 15 predições: {predicoes[:15]}")
        
        # Criamos uma nova coluna no DataFrame com os resultados da predição.
        df_db['relevante_pred'] = predicoes
        
        print("\nDataFrame final com a coluna de predição 'relevante_pred':")
        print(df_db[['pncp_id', 'objeto_compra', 'relevante_pred']].head())
        print("\nContagem de predições (0s vs 1s) no banco de dados inteiro:")
        print(df_db['relevante_pred'].value_counts())

        # --- 4c. FILTRAGEM ---
        print(f"\n--- ETAPA 4c: Filtrando o DataFrame para manter APENAS os relevantes (predição == 1) ---")
        
        # Esta é uma filtragem booleana do pandas.
        # 'df_db['relevante_pred'] == 1' cria uma série de True/False.
        # Passar isso para df_db[...] retorna SÓ as linhas onde era True.
        df_relevante = df_db[df_db['relevante_pred'] == 1].copy()
        
        # Inspecionando o DataFrame filtrado:
        print("Filtragem concluída.")
        print(f"Tipo da variável 'df_relevante': {type(df_relevante)}")
        print(f"Formato (shape) do DataFrame filtrado: {df_relevante.shape}")
        
        print(f"\n--- RESULTADO DA FILTRAGEM ---")
        print(f"De {len(df_db)} itens totais, {len(df_relevante)} foram classificados como RELEVANTES.")

        # --- 4d. SALVAMENTO DO RESULTADO ---
        print(f"\n--- ETAPA 4d: Salvando o DataFrame filtrado em '{ARQUIVO_SAIDA_FILTRADO}' ---")
        
        # Salva o DataFrame SÓ com os relevantes em um novo arquivo CSV.
        # 'index=False': Não salva o índice do pandas no arquivo.
        # 'encoding='utf-8-sig'': Garante que acentuação funcione bem,
        #                         especialmente ao abrir no Excel.
        df_relevante.to_csv(ARQUIVO_SAIDA_FILTRADO, index=False, encoding='utf-8-sig')
        
        print(f"\nArquivo '{ARQUIVO_SAIDA_FILTRADO}' salvo com sucesso!")
        
        # Mostra uma amostra final do que foi salvo.
        print("\nAmostra dos dados RELEVANTES encontrados e salvos (primeiras 10 linhas):")
        print(df_relevante[['pncp_id', 'objeto_compra']].head(10))

    else:
        # Este 'else' é ativado se 'len(df_db) == 0'
        print("ALERTA: O banco de dados está vazio. Nenhuma licitação foi carregada.")
        print("Rode o script de coleta (PASSO 2) primeiro.")

# Bloco 'except' para o caso do arquivo de modelo .joblib não ser encontrado.
except FileNotFoundError:
    print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    print(f"ERRO CRÍTICO: Modelo '{NOME_ARQUIVO_MODELO}' não encontrado.")
    print(f"Certifique-se de rodar a CÉLULA 3 (Treinamento) primeiro.")
    print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

except Exception as e:
    # Captura qualquer outro erro inesperado (ex: a função 'preprocessar_texto' não foi definida)
    print(f"Ocorreu um erro inesperado: {e}")
    print(f"Verifique se a CÉLULA 2 ('preprocessar_texto') foi rodada.")

print("\n--- Célula 4 concluída ---")

--- INICIANDO APLICAÇÃO DO MODELO ---
Modelo a ser carregado: 'modelo_relevancia.joblib'
Banco de dados de origem: 'pncp_data.db'
Arquivo de saída: 'licitacoes_filtradas.csv'

--- ETAPA 2: Carregando modelo 'modelo_relevancia.joblib' do disco ---
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
ERRO CRÍTICO: Modelo 'modelo_relevancia.joblib' não encontrado.
Certifique-se de rodar a CÉLULA 3 (Treinamento) primeiro.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

--- Célula 4 concluída ---
